In [1]:
# Initial imports
import pandas as pd
import alpaca_trade_api as tradeapi
from finta import TA
import numpy as np


In [2]:
# API credentials
ALPACA_API_KEY = 'PKETI0Q5C8PPUXVNHJFJ'
ALPACA_SECRET_KEY = 'WQxmEpBCoMXydCCg0G8cUe6hGLpgaAfrvedZ09Fy'
ALPACA_API_BASE_URL = "https://paper-api.alpaca.markets"

# Create a connection to the API 
api = tradeapi.REST(ALPACA_API_KEY, ALPACA_SECRET_KEY, ALPACA_API_BASE_URL, api_version="v2")

# Set signal variable
signal = 1

# Create buy signal, num shares and ticker
if signal == 1:
    orderSide = "buy"
else:
    orderSide = "sell"
    
# Set the ticket symbol and the number of shares to buy
ticker = "TSLA"
number_of_shares = 1

# Make API call
prices = api.get_bars(ticker, "5Min", "2022-08-22", "2022-10-14", adjustment='raw').df
# Reorganize the DataFrame
prices = pd.concat([prices], axis=1, keys=["TSLA"])

# Drop the Multi-Index from the DataFrame
prices.columns = prices.columns.droplevel(0)
# create a seperate dataframe for signals
signals_df = prices
signals_df

,open,high,low,close,volume,trade_count,vwap
timestamp,,,,,,,
2022-08-22 05:55:00+00:00,884.50,884.50,884.50,884.50,105,3,884.504762
2022-08-22 08:00:00+00:00,875.95,877.95,867.99,867.99,19561,1203,872.356079
2022-08-22 08:05:00+00:00,868.33,871.00,868.33,870.50,20461,923,870.052425
2022-08-22 08:10:00+00:00,870.10,870.60,869.00,870.60,7189,480,869.877215
2022-08-22 08:15:00+00:00,869.11,871.00,869.00,871.00,7821,545,870.159190
...,...,...,...,...,...,...,...
2022-10-14 23:35:00+00:00,204.50,204.50,204.17,204.42,10743,340,204.383027
2022-10-14 23:40:00+00:00,204.40,204.71,204.30,204.32,11217,393,204.469150
2022-10-14 23:45:00+00:00,204.32,204.65,204.31,204.35,7935,419,204.405887


In [3]:
#  Setup EMAs for crosses
signals_df["9EMA"] = TA.EMA(signals_df, 9)
signals_df["20EMA"] = TA.EMA(signals_df, 20)
signals_df["50EMA"] = TA.EMA(signals_df, 50)
signals_df["200SMA"] = TA.SMA(signals_df, 200)

# Setup Indicators
signals_df["ATR"] = TA.ATR(signals_df)
bbands_df = TA.BBANDS(signals_df)
macd_df = TA.MACD(signals_df)
signals_df["OBV"] = TA.OBV(signals_df)
signals_df["RSI"] = TA.RSI(signals_df)

# join macd and bbands Dataframes to signals_df
bbands_df = pd.concat([bbands_df, macd_df], axis=1)
signals_df = pd.concat([signals_df, bbands_df], axis=1)
signals_df.drop(columns="SIGNAL", inplace=True)
signals_df

# Position Relative to BollingerBands, Above Upper_BB = 1, Below Lower_BB = -1, Inbetween both = 0
signals_df["Bollinger_Bands_Position"] = 0.0
# 9EMA is above 20EMA = 1
signals_df["9EMA/20EMA_Cross"] = 0.0
# 50 EMA is above 200SMA = 1
signals_df["50EMA/200SMA_Cross"] = 0.0
# MACD Increasing = 1, Decreasing = -1
signals_df["MACD_Rate"] = 0.0
# OBV is positive = 1, OBV is 
signals_df["OBV_Zero_Cross"] = 0.0
# If RSI is above 70: +1, If RSI below 30: -1
signals_df["RSI_Position"] = 0.0
# Price is crossing above VWAP = 1
signals_df["VWAP_Cross"] = 0.0

# Review DataFrame
signals_df

,open,high,low,close,volume,trade_count,vwap,9EMA,20EMA,50EMA,...,BB_MIDDLE,BB_LOWER,MACD,Bollinger_Bands_Position,9EMA/20EMA_Cross,50EMA/200SMA_Cross,MACD_Rate,OBV_Zero_Cross,RSI_Position,VWAP_Cross
timestamp,,,,,,,,,,,,,,,,,,,,,
2022-08-22 05:55:00+00:00,884.50,884.50,884.50,884.50,105,3,884.504762,884.500000,884.500000,884.500000,...,NaN,NaN,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-08-22 08:00:00+00:00,875.95,877.95,867.99,867.99,19561,1203,872.356079,875.327778,875.832250,876.079900,...,NaN,NaN,-0.370417,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-08-22 08:05:00+00:00,868.33,871.00,868.33,870.50,20461,923,870.052425,873.349180,873.874280,874.145049,...,NaN,NaN,-0.394149,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-08-22 08:10:00+00:00,870.10,870.60,869.00,870.60,7189,480,869.877215,872.417886,872.929046,873.204916,...,NaN,NaN,-0.391673,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-08-22 08:15:00+00:00,869.11,871.00,869.00,871.00,7821,545,870.159190,871.996097,872.462426,872.727963,...,NaN,NaN,-0.366165,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-10-14 23:35:00+00:00,204.50,204.50,204.17,204.42,10743,340,204.383027,204.242027,204.233724,205.005395,...,204.1715,203.935367,-0.083409,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-10-14 23:40:00+00:00,204.40,204.71,204.30,204.32,11217,393,204.469150,204.257621,204.241941,204.978517,...,204.1825,203.939980,-0.069355,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-10-14 23:45:00+00:00,204.32,204.65,204.31,204.35,7935,419,204.405887,204.276097,204.252232,204.953869,...,204.1950,203.944737,-0.055160,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
# Generate the trading signals 1 (entry) or -1 (exit) for a long position trading algorithm
# where -1 is when the Close price is less than the BB_LOWER window
# where 1 is when the Close price is greater the the BB_UPPER window
for index, row in signals_df.iterrows():
    if row["close"] < row["BB_LOWER"]:
        signals_df.loc[index, "Bollinger_Bands_Position"] = -1.0
    if row["close"] > row["BB_UPPER"]:
        signals_df.loc[index,"Bollinger_Bands_Position"] = 1.0

# Generate the trading signal 1 or 0
# where 1 is when the Short window is greater than (or crosses over) the Long Window
# where 0 is when the Short window is under the Long window
signals_df["9EMA/20EMA_Cross"][9:] = np.where(
    signals_df["9EMA"][9:] > signals_df["20EMA"][9:], 1.0, 0.0)
# Calculate the points in time at which a position should be taken, 1 or -1, when there is a cross
signals_df["9EMA/20EMA_Cross"] = signals_df["9EMA/20EMA_Cross"].diff()

# Generate the trading signal 1 or 0,
# where 1 is when the Short window is greater than (or crosses over) the Long Window
# where 0 is when the Short window is under the Long window
signals_df["50EMA/200SMA_Cross"][50:] = np.where(
    signals_df["50EMA"][50:] > signals_df["200SMA"][50:], 1.0, 0.0)
# Calculate the points in time at which a position should be taken, 1 or -1, when the 50EMA Crosses the 200SMA
signals_df["50EMA/200SMA_Cross"] = signals_df["50EMA/200SMA_Cross"].diff()

# WORK IN PROGRESS - WILL ADD SOON
# # Generate the trading signal 1 or 0,
# # where 1 is when the MACD is Increasing
# # where 0 is when the MACD is Decreasing
# n = signals_df["MACD_Rate"].index
# signals_df["MACD_Rate"] = np.where(
#     signals_df["MACD"][n+1] > signals_df["MACD"][n], 1.0, 0.0)
# # Calculate the points in time at which a position should be taken, 1 or -1
# signals_df["MACD_Rate"] = signals_df["MACD_Rate"].diff()

# Generate the trading signal 1 or 0,
# where 1 is when the OBV is Positive
# where 0 is when the OBV is Negative
signals_df["OBV_Zero_Cross"] = np.where(
    signals_df["OBV"] > 0, 1.0, 0.0)
# Calculate the points in time at which a position should be taken, 1 or -1, when OBV crosses the zero-line
signals_df["OBV_Zero_Cross"] = signals_df["OBV_Zero_Cross"].diff()

# Generate the trading signals 1 (entry) or -1 (exit) for a long position trading algorithm
# where -1 is when the RSI is below 30
# where 1 is when the RSI is above 70
for index, row in signals_df.iterrows():
    if 30 > row["RSI"]:
        signals_df.loc[index, "RSI_Position"] = -1.0
    if 70 < row["RSI"]:
        signals_df.loc[index,"RSI_Position"] = 1.0

# Generate the trading signal 1 or 0,
# where 1 is when the price is above VWAP
# where 0 is when the price is below VWAP
signals_df["VWAP_Cross"] = np.where(
    signals_df["vwap"] < signals_df["close"], 1.0, 0)
# Calculate the points in time at which a position should be taken, 1 or -1, when price crosses VWAP
signals_df["VWAP_Cross"] = signals_df["VWAP_Cross"].diff()

signals_df

C:\Users\range\anaconda3\envs\dev\envs\dev\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\range\anaconda3\envs\dev\envs\dev\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,open,high,low,close,volume,trade_count,vwap,9EMA,20EMA,50EMA,...,BB_MIDDLE,BB_LOWER,MACD,Bollinger_Bands_Position,9EMA/20EMA_Cross,50EMA/200SMA_Cross,MACD_Rate,OBV_Zero_Cross,RSI_Position,VWAP_Cross
timestamp,,,,,,,,,,,,,,,,,,,,,
2022-08-22 05:55:00+00:00,884.50,884.50,884.50,884.50,105,3,884.504762,884.500000,884.500000,884.500000,...,NaN,NaN,0.000000,0.0,NaN,NaN,0.0,NaN,0.0,NaN
2022-08-22 08:00:00+00:00,875.95,877.95,867.99,867.99,19561,1203,872.356079,875.327778,875.832250,876.079900,...,NaN,NaN,-0.370417,0.0,0.0,0.0,0.0,0.0,-1.0,0.0
2022-08-22 08:05:00+00:00,868.33,871.00,868.33,870.50,20461,923,870.052425,873.349180,873.874280,874.145049,...,NaN,NaN,-0.394149,0.0,0.0,0.0,0.0,1.0,-1.0,1.0
2022-08-22 08:10:00+00:00,870.10,870.60,869.00,870.60,7189,480,869.877215,872.417886,872.929046,873.204916,...,NaN,NaN,-0.391673,0.0,0.0,0.0,0.0,0.0,-1.0,0.0
2022-08-22 08:15:00+00:00,869.11,871.00,869.00,871.00,7821,545,870.159190,871.996097,872.462426,872.727963,...,NaN,NaN,-0.366165,0.0,0.0,0.0,0.0,0.0,-1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-10-14 23:35:00+00:00,204.50,204.50,204.17,204.42,10743,340,204.383027,204.242027,204.233724,205.005395,...,204.1715,203.935367,-0.083409,1.0,1.0,0.0,0.0,0.0,0.0,0.0
2022-10-14 23:40:00+00:00,204.40,204.71,204.30,204.32,11217,393,204.469150,204.257621,204.241941,204.978517,...,204.1825,203.939980,-0.069355,0.0,0.0,0.0,0.0,0.0,0.0,-1.0
2022-10-14 23:45:00+00:00,204.32,204.65,204.31,204.35,7935,419,204.405887,204.276097,204.252232,204.953869,...,204.1950,203.944737,-0.055160,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
# here we create the exit column, our "y", for use in supervised ML
# How many rows are in the signals_df? for use in modifying DataFrame
periods = signals_df.shape[0]
# Exit is the labeled target for ML, Exit Price is for use in Pnl Metrics
signals_df["Exit"] = 0
signals_df["Exit Price"] = 0

# Exit is the labeled target for ML, Exit Price is for use in Pnl Metrics
signals_df["Exit"] = 0
signals_df["Exit Price"] = 0

# we also figure out our exit price
# target price signaling a win will be 1
# stop price signaling a loss will be -1
# loop thru the dataframe, from row 200 to the end (periods)
for j in range(200, periods):
    # entries will be on candle close
    entry = signals_df["close"].iloc[j]
    # calculate volatility for each candle
    atr = signals_df["ATR"].iloc[j]
    # stop is entry price minus the average volatility for the entry period
    stop = entry - atr
    # target is entry price plus the average volatility for the entry period times a multiplier
    target = entry + (3 * atr)
    # loop again thru the dataset to compare j entry price to future closing prices to see if we hit target or stop
    for k in range(j + 1, periods):
        # current low of the candle
        curr_low = signals_df["low"].iloc[k]
        # current high of the candle
        curr_high = signals_df["high"].iloc[k]
        # record and break if we hit stop or target, if not we check the next k period
        # if current low breaks our stop we should've sold: -1 in our "Exit" column
        if curr_low <= stop:
            signals_df["Exit Price"].iloc[j] = stop
            signals_df["Exit"].iloc[j] = -1
            # if we hit the stop break the inner loop to check the next row
            break
        # if current high breaks our target we should've sold: +1 in our "Exit" column
        elif curr_high >= target:
            signals_df["Exit Price"].iloc[j] = target
            signals_df["Exit"].iloc[j] = 1
            # if we hit the target break the inner loop to check the next row
            break

# drop beginning columns to avoid NaN values from EMA/SMA calculations
signals_df = signals_df[200:]

signals_df

C:\Users\range\anaconda3\envs\dev\envs\dev\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,open,high,low,close,volume,trade_count,vwap,9EMA,20EMA,50EMA,...,MACD,Bollinger_Bands_Position,9EMA/20EMA_Cross,50EMA/200SMA_Cross,MACD_Rate,OBV_Zero_Cross,RSI_Position,VWAP_Cross,Exit,Exit Price
timestamp,,,,,,,,,,,,,,,,,,,,,
2022-08-23 09:40:00+00:00,876.14,876.15,876.14,876.15,534,25,876.101067,876.308485,875.618192,873.395106,...,1.127085,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-1,875.281429
2022-08-23 09:45:00+00:00,877.00,877.15,877.00,877.15,1396,78,877.086711,876.476788,875.764079,873.542402,...,1.118015,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1,876.269286
2022-08-23 09:50:00+00:00,877.19,877.19,877.00,877.00,430,50,877.088744,876.581430,875.881785,873.678035,...,1.086203,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,-1,876.198571
2022-08-23 09:55:00+00:00,877.80,877.80,877.80,877.80,593,40,877.908482,876.825144,876.064473,873.839727,...,1.112717,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1,876.994286
2022-08-23 10:00:00+00:00,878.10,878.10,877.59,877.59,3081,129,877.867299,876.978115,876.209761,873.986836,...,1.104058,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1,876.776429
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-10-14 23:35:00+00:00,204.50,204.50,204.17,204.42,10743,340,204.383027,204.242027,204.233724,205.005395,...,-0.083409,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0.000000
2022-10-14 23:40:00+00:00,204.40,204.71,204.30,204.32,11217,393,204.469150,204.257621,204.241941,204.978517,...,-0.069355,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0,0.000000
2022-10-14 23:45:00+00:00,204.32,204.65,204.31,204.35,7935,419,204.405887,204.276097,204.252232,204.953869,...,-0.055160,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.000000


In [6]:
# check if there is potentially-unwanted zeros in the dataframe
signals_df["Exit"].value_counts()

-1    5477
 1    1870
 0      20
Name: Exit, dtype: int64

In [7]:
# Remove all unwanted zeros from the exit column
signals_df = signals_df.loc[signals_df["Exit"] != 0]
signals_df["Exit"].value_counts()

-1    5477
 1    1870
Name: Exit, dtype: int64

In [8]:
# Create a new dataframe to hold time-periods where a signal occured
column_names = signals_df.columns
df = pd.DataFrame(columns = column_names)

# If a bollinger-bands signal occurred, add that row to our dataframe
for index, row in signals_df.iterrows(): 
    if signals_df["Bollinger_Bands_Position"].loc[index] == 1:
        df.loc[index] = signals_df.loc[index]
    if signals_df["Bollinger_Bands_Position"].loc[index] == -1:
        df.loc[index] = signals_df.loc[index]
        
# If a 9EMA/20EMA cross signal occurred, add that row to our dataframe
for index, row in signals_df.iterrows(): 
    if signals_df["9EMA/20EMA_Cross"].loc[index] == 1:
        df.loc[index] = signals_df.loc[index]
    if signals_df["9EMA/20EMA_Cross"].loc[index] == -1:
        df.loc[index] = signals_df.loc[index]
        
# If a 50EMA/200SMA cross signal occurred, add that row to our dataframe
for index, row in signals_df.iterrows(): 
    if signals_df["50EMA/200SMA_Cross"].loc[index] == 1:
        df.loc[index] = signals_df.loc[index]
    if signals_df["50EMA/200SMA_Cross"].loc[index] == -1:
        df.loc[index] = signals_df.loc[index]
        
# If an OBV_Zero_Cross signal occurred, add that row to our dataframe
for index, row in signals_df.iterrows(): 
    if signals_df["OBV_Zero_Cross"].loc[index] == 1:
        df.loc[index] = signals_df.loc[index]
    if signals_df["OBV_Zero_Cross"].loc[index] == -1:
        df.loc[index] = signals_df.loc[index]
        
# If a RSI_Position signal occurred, add that row to our dataframe
for index, row in signals_df.iterrows(): 
    if signals_df["RSI_Position"].loc[index] == 1:
        df.loc[index] = signals_df.loc[index]
    if signals_df["RSI_Position"].loc[index] == -1:
        df.loc[index] = signals_df.loc[index]
        
# If a VWAP_Cross signal occurred, add that row to our dataframe
for index, row in signals_df.iterrows(): 
    if signals_df["VWAP_Cross"].loc[index] == 1:
        df.loc[index] = signals_df.loc[index]
    if signals_df["VWAP_Cross"].loc[index] == -1:
        df.loc[index] = signals_df.loc[index]
        
df

,open,high,low,close,volume,trade_count,vwap,9EMA,20EMA,50EMA,...,MACD,Bollinger_Bands_Position,9EMA/20EMA_Cross,50EMA/200SMA_Cross,MACD_Rate,OBV_Zero_Cross,RSI_Position,VWAP_Cross,Exit,Exit Price
2022-08-23 11:25:00+00:00,875.1388,875.1500,873.7200,874.0000,11667.0,448.0,874.428145,875.260842,875.617281,874.662040,...,-0.121410,-1.0,0.0,0.0,0.0,0.0,0.0,-1.0,1.0,875.937143
2022-08-23 13:00:00+00:00,873.5600,873.9266,872.1400,872.1400,14634.0,930.0,872.964875,874.275653,874.976062,874.939863,...,-0.564429,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,876.398179
2022-08-23 13:35:00+00:00,873.4100,875.8500,868.2800,868.5350,654670.0,16590.0,872.071673,872.476592,873.587233,874.297436,...,-1.060873,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,866.627964
2022-08-23 13:40:00+00:00,868.6835,870.7400,863.7700,863.8771,529130.0,14923.0,867.748500,870.756694,872.662459,873.888774,...,-1.699305,-1.0,0.0,0.0,0.0,0.0,-1.0,0.0,1.0,870.722607
2022-08-23 14:10:00+00:00,878.9300,885.6900,878.6587,883.8100,718749.0,17369.0,882.429254,875.457479,874.310715,874.322413,...,0.536906,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,897.682129
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-10-14 19:00:00+00:00,208.0000,208.5900,207.9500,208.2700,907910.0,11567.0,208.339939,208.047633,208.349447,210.185106,...,-0.509784,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,207.679236
2022-10-14 21:20:00+00:00,204.5700,204.5700,204.4000,204.5500,7975.0,544.0,204.504672,204.608348,205.012378,206.700548,...,-0.608532,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-1.0,204.351429
2022-10-14 23:10:00+00:00,204.1900,204.1900,204.0500,204.1000,5787.0,316.0,204.106401,204.131088,204.207023,205.169425,...,-0.184282,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,1.0,204.637857
2022-10-14 23:15:00+00:00,204.1000,204.2000,204.0700,204.2000,6314.0,345.0,204.160509,204.144870,204.206355,205.131408,...,-0.165547,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-1.0,204.021429


In [9]:
# choose if you want continuous or discrete features
continuous_features = ["volume", "trade_count", "vwap", "9EMA", "20EMA", "50EMA", "200SMA", "ATR", "OBV", "RSI", "BB_UPPER", "BB_MIDDLE", "BB_LOWER", "MACD"]
discrete_features = ["Bollinger_Bands_Position", "9EMA/20EMA_Cross", "50EMA/200SMA_Cross", "OBV_Zero_Cross", "RSI_Position", "VWAP_Cross"]
X = df[continuous_features]

# 1 means a buy would've produced a profit, -1 means a sale would've produced a profit
y = df["Exit"]
# X
y

2022-08-23 11:25:00+00:00    1.0
2022-08-23 13:00:00+00:00    1.0
2022-08-23 13:35:00+00:00   -1.0
2022-08-23 13:40:00+00:00    1.0
2022-08-23 14:10:00+00:00    1.0
                            ... 
2022-10-14 19:00:00+00:00   -1.0
2022-10-14 21:20:00+00:00   -1.0
2022-10-14 23:10:00+00:00    1.0
2022-10-14 23:15:00+00:00   -1.0
2022-10-14 23:20:00+00:00    1.0
Name: Exit, Length: 4730, dtype: float64

In [10]:
# check for potentially unwanted zeros
df["Exit"].value_counts()

-1.0    3534
 1.0    1196
Name: Exit, dtype: int64

In [ ]:
# Note: check if we have duplicate rows in the dataframe